<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Parking_spot_detection_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parking Spot Detector 1


###Notice:
Since the model detects 3 different parking slots, it is difficult to detect them and if you want to continue this in the future, you must create a mask for each dataset and train on the images with some masks you created not the masks in the dataset, since they are a few and not accuracte enough,
 or you can combine this two projects to get a better project of them all.


In [ ]:
import os
import cv2
import json
import pickle
import shutil
import sklearn
import skimage
import zipfile
import IPython
import scipy.io
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from sklearn import svm
import tensorflow_datasets
from sklearn import metrics
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn import model_selection

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d blanderbuss/parking-lot-dataset
!unzip \*.zip && rm *.zip
IPython.display.clear_output()

##Image Classifier
At first we need to create a model to predict if a spot shown in the image is Empty or Occupied.

We use the [**parking lot dataset**](https://www.kaggle.com/datasets/blanderbuss/parking-lot-dataset) from kaggle for both classification and detection tasks.

###Data

In [ ]:
cls_data = './PKLot/PKLotSegmented'
train_path = './data/train'
val_path   = './data/validation'
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

VAL_THRESHOLD = 0.2

In [ ]:
#since the models are big so we need to select only a portion of the data so the ram won't get fulled
limit = 80
data_arr = []

for cat in os.listdir(cls_data):
  for crs in os.listdir(cls_data+'/'+cat):
    for day in os.listdir(cls_data+'/'+cat+'/'+crs):
      for E_O in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day):
        for img in os.listdir(cls_data+'/'+cat+'/'+crs+'/'+day+'/'+E_O)[:limit]:
          path = os.path.join(cls_data, cat, crs, day, E_O, img)
          folder_name = train_path if np.random.randn()<VAL_THRESHOLD else val_path
          target = os.path.join(folder_name, img)
          label = 0 if E_O == 'Empty' else 1
          data_arr.append([target, label])
          os.replace(path, target)


data = pd.DataFrame(data_arr, columns=['path', 'label'])
data

In [ ]:
data.groupby(['label']).count()

In [ ]:
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory('./data',
                                                               validation_split=0.2,
                                                               subset='both',
                                                               seed=32,
                                                               image_size=(150, 150),
                                                               batch_size=32)

### Visualization

In [ ]:
def show_images(ds, augmentation=False):
  plt.figure(figsize=(3, 3))
  for images, labels in ds.take(1):
      for i in range(9):
          ax = plt.subplot(3, 3, i + 1)
          if augmentation:
            images = data_augmentation(images)
            i = 0
          plt.imshow(images[i].numpy().astype("uint8"))
          plt.title(int(labels[i]))
          plt.axis("off")

In [ ]:
show_images(train_ds)